In [1]:
def extract_features(directory):
    model = VGG16()
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    model.summary()
    features = dict()
    for name in listdir(directory):
        filename = directory + '/' + name
        image = load_img(filename, target_size = (224,224))
        image = img_to_array(image)
        image = image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))
        image = preprocess_input(image)
        feature = model.predict(image, verbose=0)
        image_id = name.split('.')[0]
        features[image_id] = feature
        print('>%s' %name)
    return features

In [2]:
from os import listdir
from os import path
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

Using TensorFlow backend.


In [3]:
directory = 'arvind'
features = extract_features(directory)
print('Extracted Features: %d' %len(features))
dump(features, open('features.pkl','wb'))

W0709 20:27:13.255772 139847449843520 module_wrapper.py:136] From /home/arvind/.local/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0709 20:27:13.340297 139847449843520 module_wrapper.py:136] From /home/arvind/.local/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [4]:
def load_doc(filename):
    file = open(filename,'r')
    text = file.read()
    file.close()
    return text

In [5]:
filename = 'Flickr8k_text/arvind_doc.txt'
doc = load_doc(filename)

In [6]:
def load_descriptions(doc):
    mapping = dict()
    for line in doc.split('\n'):
        tokens = line.split()
        if len(line)<2:
            continue
        
        image_id, image_desc = tokens[0], tokens[1:]
        image_id = image_id.split('.')[0]
        image_desc = ' '.join(image_desc)
        if image_id not in mapping:
            mapping[image_id] = list()
        mapping[image_id].append(image_desc)
    return mapping 

In [7]:
descriptions = load_descriptions(doc)
print('Loaded: %d' %len (descriptions))

Loaded: 10


In [12]:
import re, string
def clean_descriptions(descriptions):
    re_punc = re.compile('[%s]' %re.escape(string.punctuation))
    for key, desc_list in descriptions.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]
            desc = desc.split()
            desc = [word.lower() for word in desc]
            desc = [re_punc.sub('',w) for w in desc]
            desc = [word for word in desc if len(word)>1]
            desc = [word for word in desc if word.isalpha()]
            desc_list[i] = ' '.join(desc)
clean_descriptions(descriptions)

In [13]:
def to_vocabulary(descriptions):
    all_desc = set()
    for key in descriptions.keys():
        [all_desc.update(d.split()) for d in descriptions[key]]
    return all_desc

vocabulary = to_vocabulary(descriptions)
print('Vocabulary Size: %d' %len(vocabulary))

Vocabulary Size: 174


In [16]:
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
    data = '\n'.join(lines)
    file = open(filename,'w')
    file.write(data)
    file.close()
save_descriptions(descriptions, 'descriptions.txt')

In [17]:
def load_set(filename):
    doc = load_doc(filename)
    dataset = list()
    for line in doc.split('\n'):
        if len(line)<1:
            continue
        identifier = line.split('.')[0]
        dataset.append(identifier)
    return set(dataset)

In [28]:
def load_clean_descriptions(filename, dataset):
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        tokens = line.split()
        image_id, image_desc = tokens[0], tokens[1:]
        if image_id in dataset:
            if image_id not in descriptions:
                descriptions[image_id] = list()
            
            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
            descriptions[image_id].append(desc)
    return descriptions

In [29]:
def load_photo_features(filename, dataset):
    all_features = load(open(filename, 'rb'))
    features = {k: all_features[k] for k in dataset}
    return features

In [30]:
from pickle import load

In [31]:
filename = 'Flickr8k_text/arvind_doc_train.txt'
train = load_set(filename)

In [32]:
print('Dataset: %d' %len(train))

Dataset: 8


In [33]:
train_descriptions = load_clean_descriptions('descriptions.txt',train)

In [34]:
print('Descriptions: train=%d'%len(train_descriptions))

Descriptions: train=8


In [35]:
train_features = load_photo_features('features.pkl', train)

In [36]:
print('Photos: train=%d' %len(train_features))

Photos: train=8


In [37]:
train_descriptions

{'1000268201_693b08cb0e': ['startseq child in pink dress is climbing up set of stairs in an entry way endseq',
  'startseq girl going into wooden building endseq',
  'startseq little girl climbing into wooden playhouse endseq',
  'startseq little girl climbing the stairs to her playhouse endseq',
  'startseq little girl in pink dress going into wooden cabin endseq'],
 '1001773457_577c3a7d70': ['startseq black dog and spotted dog are fighting endseq',
  'startseq black dog and tricolored dog playing with each other on the road endseq',
  'startseq black dog and white dog with brown spots are staring at each other in the street endseq',
  'startseq two dogs of different breeds looking at each other on the road endseq',
  'startseq two dogs on pavement moving toward each other endseq'],
 '1002674143_1b742ab4b8': ['startseq little girl covered in paint sits in front of painted rainbow with her hands in bowl endseq',
  'startseq little girl is sitting in front of large painted rainbow endse

In [38]:
def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

In [41]:
from keras.preprocessing.text import Tokenizer
def create_tokenizer(descriptions):
    lines = to_lines(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [43]:
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' %vocab_size)

Vocabulary Size: 146


In [54]:
def create_sequences(tokenizer, max_length, descriptions, photos):
    X1, X2, y = list(), list(), list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            seq = tokenizer.texts_to_sequences([desc])[0]
            for i in range(1,len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                X1.append(photos[key][0])
                X2.append(in_seq)
                y.append(out_seq)
    return array(X1), array(X2), array(y)

In [45]:
def max_length(descriptions):
    lines = to_lines(descriptions)
    return max(len(d.split()) for d in lines)

In [69]:
def define_model(vocab_size, max_length):
    inputs1 = Input(shape = (4096,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation = 'relu')(fe1)
    inputs2 = Input(shape = (max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    decoder1 = add([fe2,se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation="softmax")(decoder2)
    model = Model(inputs=[inputs1,inputs2], outputs = outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.summary()
    plot_model(model, to_file='model.png',show_shapes=True)
    return model

In [50]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [71]:
model.fit([X1train,X2train],ytrain,epochs=20,verbose=2,callbacks=[checkpoint],validation_data = ([X1test, X2test], ytest))

Train on 454 samples, validate on 63 samples
Epoch 1/20
 - 3s - loss: 5.2773 - val_loss: 4.5468

Epoch 00001: val_loss improved from inf to 4.54677, saving model to model.h5
Epoch 2/20
 - 1s - loss: 4.3097 - val_loss: 4.3615

Epoch 00002: val_loss improved from 4.54677 to 4.36155, saving model to model.h5
Epoch 3/20
 - 1s - loss: 3.9765 - val_loss: 4.3566

Epoch 00003: val_loss improved from 4.36155 to 4.35664, saving model to model.h5
Epoch 4/20
 - 1s - loss: 3.6636 - val_loss: 4.2699

Epoch 00004: val_loss improved from 4.35664 to 4.26994, saving model to model.h5
Epoch 5/20
 - 1s - loss: 3.5725 - val_loss: 4.3115

Epoch 00005: val_loss did not improve from 4.26994
Epoch 6/20
 - 1s - loss: 3.4108 - val_loss: 4.1985

Epoch 00006: val_loss improved from 4.26994 to 4.19851, saving model to model.h5
Epoch 7/20
 - 1s - loss: 3.3286 - val_loss: 4.2719

Epoch 00007: val_loss did not improve from 4.19851
Epoch 8/20
 - 1s - loss: 3.2580 - val_loss: 4.3882

Epoch 00008: val_loss did not improv

In [67]:
from numpy import array
from pickle import load
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint

In [52]:
max_length = max_length(train_descriptions)
print('Description Length: %d' %max_length)

Description Length: 19


In [55]:
X1train, X2train, ytrain = create_sequences(tokenizer, max_length, train_descriptions, train_features)

In [58]:
filename = 'Flickr8k_text/arvind_doc_dev.txt'

In [59]:
test = load_set(filename)

In [60]:
print('Dataset: %d' %len(test))

Dataset: 2


In [61]:
test_descriptions = load_clean_descriptions('descriptions.txt',test)

In [62]:
print( 'Descriptions: test = %d' %len(test_descriptions))

Descriptions: test = 2


In [63]:
test_features = load_photo_features('features.pkl', test)

In [64]:
print('Photos: test = %d' %len(test_features))

Photos: test = 2


In [65]:
X1test, X2test, ytest = create_sequences(tokenizer, max_length, test_descriptions,test_features )

In [70]:
model = define_model(vocab_size, max_length)

W0710 16:33:54.180612 139847449843520 module_wrapper.py:136] From /home/arvind/.local/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 19)           0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 19, 256)      37376       input_6[0][0]                    
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 4096)         0           input_5[0][0]                    
__________________________________________________________________________________________________
dropout_4 

In [84]:
ytest[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [85]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [86]:
def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen = max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = argmax(yhat)
        word = word_for_id(yhat,tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text

In [87]:
def cleanup_summary(summary):
    index = summary.find('startseq')
    if index > -1:
        summary = summary[len('startseq '):]
    index = summary.find(' endseq')
    if index > -1:
        summary = summary[:index]
    return summary

In [88]:
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = list(), list()
    for key, desc_list in descriptions.items():
        yhat = generate_desc(model, tokenizer, photos[key], max_length)
        yhat = cleanup_summary(yhat)
        references = [cleanup_summary(d).split() for d in desc_list]
        actual.append(references)
        predicted.append(yhat.split())
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0,0,0,0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5,0.5,0,0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3,0.3,0.3,0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25,0.25,0.25,0.25)))

In [89]:
from numpy import argmax
from pickle import load
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

In [90]:
filename = 'model.h5'
model = load_model(filename)
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)

BLEU-1: 0.416667
BLEU-2: 0.000000
BLEU-3: 0.000000
BLEU-4: 0.000000


/home/arvind/.local/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/arvind/.local/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/arvind/.local/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunc

In [91]:
from pickle import dump

In [92]:
dump(tokenizer, open('tokenizer.pkl','wb'))

In [93]:
tokenizer = load(open('tokenizer.pkl','rb'))
max_length = max_length

In [94]:
max_length

19

In [95]:
model = load_model('model.h5')

In [97]:
def extracted_features(filename):
    model = VGG16()
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    image=load_img(filename, target_size = (224,224))
    image = img_to_array(image)
    image = image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))
    image = preprocess_input(image)
    feature = model.predict(image,verbose=0)
    return feature

photo = extracted_features('example.jpg')

In [98]:
description = generate_desc(model, tokenizer, photo, max_length)
description = cleanup_summary(description)
print(description)

little little little little into into into


In [100]:
len(X2test)

63